<a href="https://colab.research.google.com/github/JIJIN-EH97/AI_ML_COURSE_ICTAK_JIJIN/blob/3rd-intermediate-exam/3rd_intermediate_exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Libraies

In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

Load data

In [7]:
train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')
submission_df = pd.read_csv('/content/sample_submission_M0L0uXE.csv')

In [8]:
print(train_df.describe())


        employee_id  no_of_trainings           age  previous_year_rating  \
count  54808.000000     54808.000000  54808.000000          50684.000000   
mean   39195.830627         1.253011     34.803915              3.329256   
std    22586.581449         0.609264      7.660169              1.259993   
min        1.000000         1.000000     20.000000              1.000000   
25%    19669.750000         1.000000     29.000000              3.000000   
50%    39225.500000         1.000000     33.000000              3.000000   
75%    58730.500000         1.000000     39.000000              4.000000   
max    78298.000000        10.000000     60.000000              5.000000   

       length_of_service  KPIs_met >80%   awards_won?  avg_training_score  \
count       54808.000000   54808.000000  54808.000000        54808.000000   
mean            5.865512       0.351974      0.023172           63.386750   
std             4.265094       0.477590      0.150450           13.371559   
min    

In [9]:
print(train_df.isnull().sum())

employee_id                0
department                 0
region                     0
education               2409
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating    4124
length_of_service          0
KPIs_met >80%              0
awards_won?                0
avg_training_score         0
is_promoted                0
dtype: int64


In [10]:
print(train_df['is_promoted'].value_counts(normalize=True))

is_promoted
0    0.91483
1    0.08517
Name: proportion, dtype: float64


Pre po

In [26]:
df = train_df.copy()
# Features & target
X = df.drop(["employee_id", "is_promoted"], axis=1)
y = df["is_promoted"]

# Preprocessing
categorical = ["department", "region", "education", "gender", "recruitment_channel"]
numeric = ["age", "length_of_service", "avg_training_score", "no_of_trainings"]

preprocessor = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore"), categorical),
    ("num", StandardScaler(), numeric)
])

# Model pipeline
model = Pipeline([
    ("preprocess", preprocessor),
    ("clf", XGBClassifier(scale_pos_weight=5, eval_metric="logloss"))
])

# Train/test split
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=0.2)

# Fit model
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_val)

# Evaluate
print("F1 Score:", f1_score(y_val, y_pred))


# Predict on test using the pipeline
X_test_processed = test_df.drop("employee_id", axis=1)
predictions = model.predict(X_test_processed)

# Update and save submission
submission_df['is_promoted'] = predictions
submission_df.to_csv('submission1.csv', index=False)

F1 Score: 0.4725848563968668


In [27]:
print("Submission saved to submission1.csv")

Submission saved to submission1.csv
